# Выбор локации для скважины

Нефтедобывающей компании нужно решить, где бурить новую скважину.
Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

В наличии датасеты с данными о пробах нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Задача - построить модель для определения региона, где добыча принесёт наибольшую прибыль. Возможную прибыль и риски нужно проанализировать техникой Bootstrap.

## Загрузка и подготовка данных

### Обзор данных

Импортируем библиотеки:

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Считаем датасеты и сохраним в переменные:

In [2]:
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_0 = pd.read_csv('geo_data_0.csv')
    data_1 = pd.read_csv('geo_data_1.csv')
    data_2 = pd.read_csv('geo_data_2.csv')

Выведем на экран первые, последние и случайные пять строк каждой таблицы, а также общую информацию. Создадим для этого собственную функцию и применим ее к списку датасетов:

In [3]:
def total_view(df, n_rows=5, seed=None):
    '''Возвращает первые, случайные и последние строки таблицы (по умолчанию по 5 строк)'''
    return pd.concat([df.head(n_rows),
                      df.sample(n_rows,random_state=seed),
                      df.tail(n_rows)
                     ])

# список датасетов
datasets_list = [data_0, data_1, data_2]

# проименуем датасеты
data_0.name = 'data_0'
data_1.name = 'data_1'
data_2.name = 'data_2'

# выведем на экран результат работы total_view(), describe() и info()  по каждому датасету
for data in datasets_list:
    display(total_view(data).style.set_caption(data.name))
    display(data.describe())
    data.info()
    print('')

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.22117,105.28
1,2acmU,1.33471,-0.340164,4.36508,73.0378
2,409Wp,1.02273,0.15199,1.41993,85.2656
3,iJLyR,-0.0321716,0.139033,2.97857,168.621
4,Xdl7t,1.98843,0.155413,4.75177,154.037
57059,vmD51,0.0791254,0.0546937,1.06314,104.171
1587,lQlTO,-0.714391,0.64966,1.50585,156.878
89360,l0GE5,0.292587,0.886486,6.26269,129.963
58001,rSU2Q,-0.105456,0.564055,2.89191,50.4604
57673,3zgrs,1.89821,0.430488,0.871471,112.594


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



,id,f0,f1,f2,product
0,kBEdx,-15.0013,-8.276,-0.00587601,3.1791
1,62mP7,14.2721,-3.47508,0.999183,26.9533
2,vyE1P,6.26319,-5.94839,5.00116,134.766
3,KcrkZ,-13.0812,-11.5061,4.99941,137.945
4,AHL4O,12.7022,-8.14743,5.00436,134.766
49411,k71zw,-4.78373,-0.816517,0.997379,30.1324
11094,dFGj8,8.05844,-10.2607,2.00498,53.9065
63220,CWtk8,-3.84427,-7.40073,1.9948,57.0856
62942,UFlnM,-4.37362,-9.63115,2.00329,57.0856
5974,J6KYj,-2.17847,-12.6985,4.99468,137.945


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



,id,f0,f1,f2,product
0,fwXo0,-1.14699,0.963328,-0.828965,27.7587
1,WJtFt,0.262778,0.269839,-2.53019,56.0697
2,ovLUW,0.194587,0.289035,-5.58643,62.8719
3,q6cA6,2.23606,-0.55376,0.930038,114.573
4,WPMUX,-0.515993,1.71627,5.89901,149.601
43255,ObjmH,-0.746891,-2.84387,2.46137,138.604
94854,koxeG,1.00347,0.949851,6.13709,143.726
29176,U4j1O,-3.17901,0.0491798,7.10036,183.499
18608,212rA,0.869148,-1.7019,4.35216,108.609
70492,sNzNa,0.776508,3.53522,-1.18805,110.634


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



В каждом из датасетов `data_0`, `data_1` и `data_2` 5 колонок и 100000 строк. Тип данных в колонках `object` и `float64`. Согласно документации к данным:
* `id` — уникальный идентификатор скважины;
* `f0`, `f1`, `f2` — три признака точек;
* `product` — объём запасов в скважине (тыс. баррелей).

Пропущенные значения отсутствуют.

### Подготовка данных

 Проверим данные на дубликаты:

In [4]:
for data in datasets_list:
    print(data.name, data.duplicated().sum())

data_0 0
data_1 0
data_2 0


Удалим колонку `id`из всех датасетов, так как там содержатся уникальные значения, не связанные с целевым признаком:

In [5]:
data_0 = data_0.drop('id', axis=1)
data_1 = data_1.drop('id', axis=1)
data_2 = data_2.drop('id', axis=1)

**Выводы:**

Каждый из датасетов `data_0`, `data_1` и `data_2` содержит данные о пробах нефти в одном из трех регионов (будем называть их в соответствии с номерами в названиях датасетов - регион 0, регион 1 и регион 2).
В строках датасетов `data_0`, `data_1` и `data_2` содержатся данные о скважинах: id скважины, три показателя качества нефти и объем запасов в скважине.

В колонке `product` содержится целевой признак. Среди признаков для обучения модели мы оставили колонки: `f0`, `f1`, `f2`. Колонку `id` удалили, так там содержатся уникальные значения, не связанные с целевым признаком

## Обучение и проверка модели

Разобьем данные на обучающую (75%) и валидационную (25%) выборки:

In [6]:
features = data_0.drop('product', axis=1)
target = data_0['product']
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features,
                                                                              target,
                                                                              test_size=0.25,
                                                                              random_state=12345)

features = data_1.drop('product', axis=1)
target = data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features,
                                                                              target,
                                                                              test_size=0.25,
                                                                              random_state=12345)

features = data_2.drop('product', axis=1)
target = data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features,
                                                                              target,
                                                                              test_size=0.25,
                                                                              random_state=12345)



print('features_train_0:',features_train_0.shape,
      'target_train_0:', target_train_0.shape,
      'features_valid_0:', features_valid_0.shape,
      'target_valid_0:', target_valid_0.shape, '\n',
      'features_train_1:',features_train_1.shape,
      'target_train_1:', target_train_1.shape,
      'features_valid_1:', features_valid_1.shape,
      'target_valid_1:', target_valid_1.shape, '\n',
      'features_train_2:',features_train_2.shape,
      'target_train_2:', target_train_2.shape,
      'features_valid_2:', features_valid_2.shape,
      'target_valid_2:', target_valid_2.shape, '\n'
     )

features_train_0: (75000, 3) target_train_0: (75000,) features_valid_0: (25000, 3) target_valid_0: (25000,) 
 features_train_1: (75000, 3) target_train_1: (75000,) features_valid_1: (25000, 3) target_valid_1: (25000,) 
 features_train_2: (75000, 3) target_train_2: (75000,) features_valid_2: (25000, 3) target_valid_2: (25000,) 



Обучим модели и сделаем предсказания на валидационных выборках:

In [7]:
def model_linear_regression(features_train, target_train, features_valid, target_valid):
    '''Обучает модель линейной регрессии, выводит на экран среднее предсказанное значение, rmse
    и возвращает предсказания.
    '''
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    print('mean: ', predictions.mean(), '\n',
          'rmse: ', mean_squared_error(target_valid, predictions) ** 0.5,
          sep=''
         )
    return predictions

In [8]:
predictions_valid_0 = model_linear_regression(features_train_0, target_train_0, features_valid_0, target_valid_0)

mean: 92.59256778438038
rmse: 37.5794217150813


In [9]:
predictions_valid_1 = model_linear_regression(features_train_1, target_train_1, features_valid_1, target_valid_1)

mean: 68.728546895446
rmse: 0.8930992867756161


In [10]:
predictions_valid_2 = model_linear_regression(features_train_2, target_train_2, features_valid_2, target_valid_2)

mean: 94.96504596800489
rmse: 40.02970873393434


**Выводы:**

Cредний запас предсказанного сырья в регионе 0 составил примерно 93 тыс. баррелей, в регионе 1 - 69 тыс. баррелей, в регионе 2 - 95 тыс. баррелей. При этом `model_0` ошибается в среднем примерно на 38 тыс. баррелей, `model_1` - на 1 тыс. баррелей, `model_2` - на 40 тыс. баррелей.

## Подготовка к расчёту прибыли

Сохраним в переменные ключевые значения (бюджет на разработку, цена за 1 тыс. баррелей, количество разрабатываемых скважин), необходимые для расчетов:

In [11]:
BUDGET = 10_000_000_000
PRICE = 450_000
COUNT = 200

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины и средний запас на 200 скважин в каждом регионе:

In [12]:
print('Breakeven point, млн. баррелей: ', np.round((BUDGET/PRICE)/1000, 3), '\n',
      'Average reserve 0, млн. баррелей: ', np.round(data_0['product'].sum() / 500000, 3), '\n',
      'Average reserve 1, млн. баррелей: ', np.round(data_1['product'].sum() / 500000, 3), '\n',
      'Average reserve 2, млн. баррелей: ', np.round(data_2['product'].sum() / 500000, 3), sep=''
     )

Breakeven point, млн. баррелей: 22.222
Average reserve 0, млн. баррелей: 18.5
Average reserve 1, млн. баррелей: 13.765
Average reserve 2, млн. баррелей: 19.0


**Выводы:**

Достаточный объём сырья для безубыточной разработки новой скважины составляет примерно 22.222 млн. баррелей (22 222 223 баррелей). Средний запас баррелей на 200 скважин в каждом регионе ниже, чем достаточный для безубыточности. Ближе всех к безубыточности по этому показателю находится регион 2, средний запас баррелей на 200 скважин в нем составляет 19 млн. баррелей. В регионе 0 этот запас составляет 18.5 млн. баррелей, а в регионе 1 - 13.765 млн. баррелей.

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [13]:
def profit(target, predictions, cnt=COUNT):
    '''Возвращает прибыль, рассчитанную по выбранным скважинам'''
    predictions_sorted = predictions.sort_values(ascending=False) 
    selected = target[predictions_sorted.index][:COUNT]
    return (PRICE * selected.sum() - BUDGET)

Напишем функцию, которая будет применять технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли, а затем рассчитывать и выводить на экран среднюю прибыль, 95%-й доверительный интервал и риск убытков для регона:

In [14]:
state = np.random.RandomState(12345)
def bootstrap(target_valid, predictions_valid, sample=500, cnt=COUNT):
    '''Рассчитывает и выводит на экран среднюю прибыль, 95%-й доверительный интервал и риск убытков для регона.'''
    values = []
    for i in range(1000):
        target_subsample = target_valid.sample(sample, replace=True, random_state=state)
        predictions_valid = pd.Series(predictions_valid, index=target_valid.index)
        predictions_subsample = predictions_valid[target_subsample.index]
        values.append(profit(target_subsample, predictions_subsample, cnt))
    values = pd.Series(values)
    risk_rate = np.round((values < 0).mean() * 100, 2)
    print('Profit mean, млн. руб.: ', np.round(values.mean() / 1_000_000, 2), '\n',
          '95% confidence interval, млн. руб.: с ', np.round(values.quantile(0.025) / 1_000_000, 2), ' по ',
          np.round(values.quantile(0.975) / 1_000_000, 2), '\n',
          'Risk rate: ', risk_rate, '%', sep=''
          )

Применим функцию `bootstrap` к валидационным данным по каждому региону:

In [15]:
# регион 0
bootstrap(target_valid_0, predictions_valid_0)

Profit mean, млн. руб.: 425.94
95% confidence interval, млн. руб.: с -102.09 по 947.98
Risk rate: 6.0%


In [16]:
# регион 1
bootstrap(target_valid_1, predictions_valid_1)

Profit mean, млн. руб.: 518.26
95% confidence interval, млн. руб.: с 128.12 по 953.61
Risk rate: 0.3%


In [17]:
# регион 2
bootstrap(target_valid_2, predictions_valid_2)

Profit mean, млн. руб.: 420.19
95% confidence interval, млн. руб.: с -115.85 по 989.63
Risk rate: 6.2%


## Выводы:

* наименьший риск и наибольшую прибыль показывает регион 1. Риск получения убытка составляет 0.3%. С вероятностью 95% прибыль в этом регионе составит от 128.12 до 953.61 млн. руб., средняя прибыль ожидается на уровне 518.26 млн. руб. Для бурения новых скважин предлагается выбрать этот регион;
* регионы 0 и 2 показывают меньшую среднюю прибыль (425.94 и 420.19 млн. руб. соответственно) и высокий риск получения убытков - около 6%, в доверительный интервал попадают убытки.